## Import modules

In [22]:
import tensorflow as tf
import keras
import pickle
from keras.models import load_model, clone_model
from keras import backend as K
from keras.callbacks import EarlyStopping
from datetime import datetime
import sklearn.preprocessing
from sklearn.metrics.pairwise import pairwise_distances
import sklearn.metrics
from molSimplifyAD.utils.pymongo_tools import connect2db, insert, count_find, convert2dataframe, push2db
from molSimplifyAD.retrain.pairing_tools import keep_lowestE
from molSimplifyAD.retrain.nets import *

In [23]:
import sys, time
import sklearn.preprocessing
from scipy import stats
import pandas as pd
import colorlover as cl
import numpy as np
from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
import plotly.io as pio
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
plotly.io.orca.config.executable = '/anaconda2/envs/pytorch/bin/orca'
init_notebook_mode(connected=True)

glob_layout = go.Layout(
    font=dict(family='Helvetica', size=24, color='black'),
    margin=dict(l=100, r=10, t=10, b=100),
    xaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=3, linewidth=3, ticklen=10,
               mirror="allticks", color="black"),
    yaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=3, linewidth=3, ticklen=10,
               mirror="allticks", color="black"),
    legend_orientation="v",
)

In [24]:
from perato_utils import is_pareto, area_under_pareto
from plot_utils import *
from gpr import process_generation_2DEI, gp_predict
from ei import getEiVec2D_aug, getPiVec2D_aug, get_ei_samples_kmedoids

## Load data

In [25]:
fnames = pickle.load(open('fnames.pkl', 'rb'))
df = pd.read_json('df_RAC155_homogap.json', orient='records', lines=True)
whole_percentage = 100
df = df.sample(n=int(whole_percentage*0.01*len(df)), random_state=0)
df = df.reset_index()

In [26]:
# The two properties to optimize.
y1l = 'alphaHOMO'
y2l = 'gap'

In [27]:
trace0 = go.Scatter(
    x=df[y1l].values,
    y=df[y2l].values,
    mode='markers',
    opacity=1,
    marker=dict(
        size=5,
        color=df['ligcharge'].values,
        colorscale='Picnic',
        colorbar=dict(
            title="ligcharge"
        ),
    ),
)
data = [trace0]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': y1l, "range": [-1.05, 0]})
layout["yaxis"].update({'title': y2l, "range": [0, 0.35]})
layout.legend.update(x=0, y=1.0, bgcolor='rgba(0,0,0,0)')
layout.update(height=500, width=600, showlegend=False)

fig = dict(data=data, layout=layout)

## Calibration set

In [28]:
df_all = df.copy()
calibration_points = df.sample(n=int(100), random_state=1234)
df = df[~df["unique_name"].isin(calibration_points['unique_name'].values)]
len(calibration_points), len(df), len(df_all)

(100, 15011, 15111)

In [29]:
tb_count = 0
tot_gen = 50 # total number of generations
points_per_gen = 10 # how many new points to add in each generation
start_percentage = 5 # percentage of points that are "known" to start with (i.e. target properties known)
points_with_ei = 1000
# We only "know" a few points at the beginning, and will "discover" points through active learning.
known_points = df.sample(n=int(start_percentage*0.01*len(df)), random_state=1234)
pareto_inds_known, pareto_points_known = is_pareto(points=known_points[[y1l, y2l]].values)
# plot_pareto_front(known_points, pareto_points_known, y1l, y2l, global_fronts=pareto_points)
print(len(known_points))
run_name = "%dpercent/PFSampling/start%d_initpoints_%d_totgen_%d_points_%d"%(whole_percentage,start_percentage, len(known_points), tot_gen, points_per_gen)
basepath = '2DEI_alphaHOMO_gap/demo/%s/' % run_name

750


## Plot global Pareto front

In [30]:
pareto_inds, pareto_points = is_pareto(points=df[[y1l, y2l]].values)
plot_pareto_front(df, pareto_points, y1l, y2l,
                  figname=basepath + 'global_pareto_front.pdf',
                  range1=[-1.05, 0], range2=[0, 0.35], show=False)

## Direct Pareto front sampling

In [ ]:
known_list = [] # points for which we know the target properties (ostensibly after a DFT calculation, but for this example we know everything already and just pretend to discover new points). Each entry is a DataFrame. New entry for each generation
pareto_list = [] # gets a new entry for each generation. Each entry is a np.array of the Pareto points
mae_list = [] # gets a new entry for each generation
gp_list = [] # gets a new entry for each generation
new_list = []
import time
start = time.time()
while tb_count < tot_gen:
    tb_count += 1
    print("=====gen-%d=====" % tb_count)
    df, gp1, gp2 = process_generation_2DEI(known_points, df, fnames, y1l, y2l)
    gp_list.append([gp1, gp2])
    df['known'] = [True if idx in list(
        known_points.index) else False for idx, row in df.iterrows()]
    known_points = df[df['known'] == True]
    unknown_points = df[df['known'] == False]
    known_list.append(known_points)
    pareto_list.append(pareto_points_known)
    df_pred = df[df['known'] == False]
    mae_list.append([np.mean(np.abs(df_pred[y1l].values-df_pred['hat_y1'].values)),
                     np.mean(np.abs(df_pred[y2l].values-df_pred['hat_y2'].values))])
    print("mae: ", mae_list[-1]) # Print out what was just appended the line before
    pareto_inds_pred, pareto_points_pred = is_pareto(
        points=df[['hat_y1', 'hat_y2']].values)
    pareto_inds_pred_unknown, pareto_points_pred_unknown = is_pareto(
        points=unknown_points[['hat_y1', 'hat_y2']].values)
    new_points_can = unknown_points.iloc[pareto_inds_pred_unknown] # The Pareto points from among the new, unknown points
    new_points_can['sigma_sum'] = new_points_can['sigma_y1'].values + new_points_can['sigma_y2'].values
    new_points = new_points_can.nlargest(points_per_gen, ['sigma_sum']) # Choose new points to discover based on sigma (uncertainty)
    new_list.append(new_points)
    print("adding %d points using kmedoids." % len(new_points)) # number of rows in the DataFrame of new points
    known_points = known_points.append(new_points[known_points.columns]) # Both new_points and known_points are Pandas DataFrames
    
    pareto_inds_known, pareto_points_known = is_pareto(
        points=known_points[[y1l, y2l]].values)
    plot_pareto_front(known_list[-1], pareto_list[-1], y1l, y2l,
                      global_fronts=pareto_points,
                      next_fronts=pareto_points_known,
                      next_points=new_points,
                      figname=basepath +
                      '/actual_pareto_gen-%d_len%d.pdf' % (tb_count, len(known_points)),
                      show=False, range1=[-1, 0], range2=[0, 0.35])
    pareto_inds_pred_new, pareto_points_pred_new = is_pareto(
        points=known_points[['hat_y1', 'hat_y2']].values)
    pareto_inds_pred_known, pareto_points_pred_known = is_pareto(
        points=known_list[-1][['hat_y1', 'hat_y2']].values)
    print("+++++prediction++++")
    plot_pareto_front(known_list[-1], pareto_points_pred_known,
                      y1l='hat_y1', y2l='hat_y2',
                      global_fronts=pareto_points_pred,
                      next_fronts=pareto_points_pred_new,
                      next_points=new_points,
                      figname=basepath +
                          'predicted_pareto_gen-%d_len%d.pdf' % (tb_count, len(known_points)),
                      show=False, range1=[-1, 0], range2=[0, 0.35])
    print(f"ellapse: {time.time() - start:.3f}s")

=====gen-1=====
mae:  [0.020661708003751725, 0.032771304964332115]
adding 10 points using kmedoids.
known_points.columns is Index(['level_0', 'index', 'RACs.D_lc-S-1-ax', 'RACs.D_lc-S-1-eq',
       'RACs.D_lc-S-2-ax', 'RACs.D_lc-S-2-eq', 'RACs.D_lc-S-3-ax',
       'RACs.D_lc-S-3-eq', 'RACs.D_lc-T-1-ax', 'RACs.D_lc-T-1-eq',
       ...
       'ligcharge', 'alphaHOMO', 'gap', 'unique_name', 'tag', 'hat_y1',
       'sigma_y1', 'hat_y2', 'sigma_y2', 'known'],
      dtype='object', length=165)
new_points is <class 'pandas.core.frame.DataFrame'>
known_points is <class 'pandas.core.frame.DataFrame'>


 <ipython-input-31-bfccc9f98eea>:33: FutureWarning:The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


+++++prediction++++
ellapse: 7.914s
=====gen-2=====
mae:  [0.020606992021133134, 0.032756672289283854]
adding 10 points using kmedoids.
known_points.columns is Index(['level_0', 'index', 'RACs.D_lc-S-1-ax', 'RACs.D_lc-S-1-eq',
       'RACs.D_lc-S-2-ax', 'RACs.D_lc-S-2-eq', 'RACs.D_lc-S-3-ax',
       'RACs.D_lc-S-3-eq', 'RACs.D_lc-T-1-ax', 'RACs.D_lc-T-1-eq',
       ...
       'ligcharge', 'alphaHOMO', 'gap', 'unique_name', 'tag', 'hat_y1',
       'sigma_y1', 'hat_y2', 'sigma_y2', 'known'],
      dtype='object', length=165)
new_points is <class 'pandas.core.frame.DataFrame'>
known_points is <class 'pandas.core.frame.DataFrame'>


 <ipython-input-31-bfccc9f98eea>:33: FutureWarning:The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


+++++prediction++++
ellapse: 14.649s
=====gen-3=====


In [11]:
with open(basepath + "known_list.pkl", 'wb') as fo:
    pickle.dump(known_list, fo)
with open(basepath + "new_list.pkl", 'wb') as fo:
    pickle.dump(new_list, fo)
# with open(basepath + "gp_list.pkl", 'wb') as fo:
#     pickle.dump(gp_list, fo)
with open(basepath + "pareto_list.pkl", 'wb') as fo:
    pickle.dump(pareto_list, fo)
with open(basepath + "df.pkl", 'wb') as fo:
    pickle.dump(df, fo)
with open(basepath + "calibration_points.pkl", 'wb') as fo:
    pickle.dump(calibration_points, fo)

In [12]:
err_y1_calibrate, uq_y1_calibrate = [], []
err_y2_calibrate, uq_y2_calibrate = [], []
for gen in range(tot_gen):
    hat_y, uq = gp_predict(gp_list[gen][0], known_list[gen], calibration_points , fnames, y1l)
    err_y1_calibrate.append(np.abs(hat_y - calibration_points[y1l].values))
    uq_y1_calibrate.append(uq)
    hat_y, uq = gp_predict(gp_list[gen][1], known_list[gen],  calibration_points , fnames, y2l)
    err_y2_calibrate.append(np.abs(hat_y - calibration_points[y2l].values))
    uq_y2_calibrate.append(uq)

In [13]:
plot_model_violin(uq_y1_calibrate, '%s-pred-std'%y1l, fillcolor='lightblue', 
                  figname=basepath+'%s_model_std.pdf'%y1l, show=False)
plot_model_violin(err_y1_calibrate, '%s-pred-err'%y1l, fillcolor='lightgrey', 
                  figname=basepath+'%s_model_err.pdf'%y1l, show=False)
plot_model_violin(uq_y2_calibrate, '%s-pred-std'%y2l, fillcolor='lightgreen', 
                  figname=basepath+'%s_model_std.pdf'%y2l, show=False)
plot_model_violin(err_y2_calibrate, '%s-pred-err'%y2l, fillcolor='lightcyan', 
                  figname=basepath+'%s_model_err.pdf'%y2l, show=False)

In [14]:
comb = 2
err_y1_calibrate, uq_y1_calibrate = [], []
err_y2_calibrate, uq_y2_calibrate = [], []
for gen in range(tot_gen):
    if gen % comb == 0:
        hat_y, uq = gp_predict(gp_list[gen][0], known_list[gen], calibration_points , fnames, y1l)
        err_y1_calibrate.append(np.abs(hat_y - calibration_points[y1l].values))
        uq_y1_calibrate.append(uq)
        hat_y, uq = gp_predict(gp_list[gen][1], known_list[gen],  calibration_points , fnames, y2l)
        err_y2_calibrate.append(np.abs(hat_y - calibration_points[y2l].values))
        uq_y2_calibrate.append(uq)

In [15]:
plot_model_violin(uq_y1_calibrate, '%s-pred-std'%y1l, fillcolor='lightblue', 
                  figname=basepath+'%s_model_std_combgen%s.pdf'%(y1l, comb), show=False)
plot_model_violin(err_y1_calibrate, '%s-pred-err'%y1l, fillcolor='lightgrey', 
                  figname=basepath+'%s_model_err_combgen%s.pdf'%(y1l, comb), show=False)
plot_model_violin(uq_y2_calibrate, '%s-pred-std'%y2l, fillcolor='lightgreen', 
                  figname=basepath+'%s_model_std_combgen%s.pdf'%(y2l, comb), show=False)
plot_model_violin(err_y2_calibrate, '%s-pred-err'%y2l, fillcolor='lightcyan', 
                  figname=basepath+'%s_model_err_combgen%s.pdf'%(y2l, comb), show=False)

In [16]:
area_global = area_under_pareto(pareto_points, pareto_points)
pareto_area_list = [area_global/area_under_pareto(pareto_list[gen], pareto_points)  for gen in range(tot_gen)]
coverage_list = [len(known_list[gen])*1./len(df) for gen in range(tot_gen)]
plot_pareto_area(coverage_list, pareto_area_list,
                 figname=basepath+'area_under_pareto.pdf', show=True)
with open(basepath + "coverage_list.pkl", 'wb') as fo:
    pickle.dump(coverage_list, fo)
with open(basepath + "pareto_area_list.pkl", 'wb') as fo:
    pickle.dump(pareto_area_list, fo)

In [17]:
pareto_pred_actual_space = df.iloc[pareto_inds_pred][[y1l, y2l]].values
df_rm1stfront = df[~df['unique_name'].isin(df.iloc[pareto_inds_pred]['unique_name'].values)]
pareto_inds_pred_2nd, pareto_points_pred_2nd = is_pareto(
        points=df_rm1stfront[['hat_y1', 'hat_y2']].values)
pareto_pred_actual_space_2nd = df_rm1stfront.iloc[pareto_inds_pred_2nd][[y1l, y2l]].values
plot_pred_actual_front(pareto_points_pred, pareto_points, 
                       pareto_pred_actual_space, pareto_pred_actual_space_2nd,
                       y1l, y2l, figname=basepath+'pred&actual_pareto_front.pdf', show=True)